Import all

In [1]:
pip install nibabel numpy scipy matplotlib scikit-image torch torchvision pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


GPU Available?

In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

2.6.0+cu126
True
NVIDIA GeForce RTX 4060 Laptop GPU


VGG16

In [ ]:
import torch
import torch.nn as nn
import nibabel as nib
import pandas as pd
import numpy as np
import scipy.ndimage
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

CSV_PATH = r"D:/ADNI Data/BETandGroups.csv"
IMG_SIZE = (128, 128, 128)
BATCH_SIZE = 3
EPOCHS = 20
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Nifti3DDataset(Dataset):
    def __init__(self, df, label_encoder):
        self.df = df.reset_index(drop=True)
        self.label_encoder = label_encoder
        self.target_shape = IMG_SIZE

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.loc[idx, "Ruta"]
        label = self.df.loc[idx, "Grupo"]

        volume = nib.load(path).get_fdata().astype(np.float32)
        zoom_factors = [t / s for t, s in zip(self.target_shape, volume.shape)]
        volume = scipy.ndimage.zoom(volume, zoom=zoom_factors, order=3)

        volume = (volume - np.mean(volume)) / (np.std(volume) + 1e-8)
        volume = np.clip(volume, -3, 3)

        volume = torch.tensor(volume).unsqueeze(0) 
        label = torch.tensor(self.label_encoder.transform([label])[0])
        return volume, label

#Arquitectura VGG16
class VGG3D(nn.Module):
    def __init__(self, num_classes=3):
        super(VGG3D, self).__init__()
        self.features = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(64, 64, kernel_size=3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(kernel_size=2, stride=2),

            nn.Conv3d(64, 128, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(128, 128, kernel_size=3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(kernel_size=2, stride=2),

            nn.Conv3d(128, 256, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(256, 256, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(256, 256, kernel_size=3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(kernel_size=2, stride=2),

            nn.Conv3d(256, 512, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, kernel_size=3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(kernel_size=2, stride=2),

            nn.Conv3d(512, 512, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, kernel_size=3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(kernel_size=2, stride=2)
        )

        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

#Cargar Datos
df = pd.read_csv(CSV_PATH)
le = LabelEncoder()
df['labels'] = le.fit_transform(df['Grupo'])

df_train, df_val = train_test_split(df, test_size=0.25, stratify=df['labels'], random_state=42)
train_dataset = Nifti3DDataset(df_train, le)
val_dataset = Nifti3DDataset(df_val, le)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

#Entrenar el modelo
model = VGG3D(num_classes=len(le.classes_)).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

for epoch in range(EPOCHS):
    model.train()
    train_loss, train_correct = 0, 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()

    acc_train = 100 * train_correct / len(train_dataset)
    print(f"Epoch {epoch+1} - Loss: {train_loss:.4f}, Train Acc: {acc_train:.2f}%")

    model.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            val_correct += (outputs.argmax(1) == labels).sum().item()
    acc_val = 100 * val_correct / len(val_dataset)
    print(f"Test Acc: {acc_val:.2f}%")

torch.save(model.state_dict(), "vgg16_3d_alzheimer.pth")
print("Modelo guardado como vgg16_3d_alzheimer.pth")


Entrenar mas epocas

In [5]:
import torch
import torch.nn as nn
import nibabel as nib
import pandas as pd
import numpy as np
import scipy.ndimage
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

CSV_PATH = r"D:/ADNI Data/BETandGroups.csv"
IMG_SIZE = (128, 128, 128)
BATCH_SIZE = 3
EPOCHS = 50
LR = 1e-5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Nifti3DDataset(Dataset):
    def __init__(self, df, label_encoder):
        self.df = df.reset_index(drop=True)
        self.label_encoder = label_encoder
        self.target_shape = IMG_SIZE

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.loc[idx, "Ruta"]
        label = self.df.loc[idx, "Grupo"]

        volume = nib.load(path).get_fdata().astype(np.float32)
        zoom_factors = [t / s for t, s in zip(self.target_shape, volume.shape)]
        volume = scipy.ndimage.zoom(volume, zoom=zoom_factors, order=3)

        volume = (volume - np.mean(volume)) / (np.std(volume) + 1e-8)
        volume = np.clip(volume, -3, 3)

        volume = torch.tensor(volume).unsqueeze(0) 
        label = torch.tensor(self.label_encoder.transform([label])[0])
        return volume, label

#Arquitectura VGG16
class VGG3D(nn.Module):
    def __init__(self, num_classes=3):
        super(VGG3D, self).__init__()
        self.features = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(64, 64, kernel_size=3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(kernel_size=2, stride=2),

            nn.Conv3d(64, 128, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(128, 128, kernel_size=3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(kernel_size=2, stride=2),

            nn.Conv3d(128, 256, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(256, 256, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(256, 256, kernel_size=3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(kernel_size=2, stride=2),

            nn.Conv3d(256, 512, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, kernel_size=3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(kernel_size=2, stride=2),

            nn.Conv3d(512, 512, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, kernel_size=3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(kernel_size=2, stride=2)
        )

        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

#Cargar Datos
df = pd.read_csv(CSV_PATH)
le = LabelEncoder()
df['labels'] = le.fit_transform(df['Grupo'])

df_train, df_val = train_test_split(df, test_size=0.25, stratify=df['labels'], random_state=42)
train_dataset = Nifti3DDataset(df_train, le)
val_dataset = Nifti3DDataset(df_val, le)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

#Entrenar el modelo
model = VGG3D(num_classes=len(le.classes_)).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, min_lr=1e-8, verbose=True)

for epoch in range(EPOCHS):
    model.train()
    train_loss, train_correct = 0, 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()

    acc_train = 100 * train_correct / len(train_dataset)
    print(f"Epoch {epoch+1} - Loss: {train_loss:.4f}, Train Acc: {acc_train:.2f}%")

    model.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            val_correct += (outputs.argmax(1) == labels).sum().item()
    acc_val = 100 * val_correct / len(val_dataset)
    print(f"Test Acc: {acc_val:.2f}%")

torch.save(model.state_dict(), "vgg16_3d_alzheimer3.pth")
print("Modelo guardado como vgg16_3d_alzheimer3.pth")


d:\ADNI Data\venv\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 1/50: 100%|██████████| 208/208 [1:16:30<00:00, 22.07s/it]


Epoch 1 - Loss: 681.4067, Train Acc: 41.03%
Test Acc: 41.15%


Epoch 2/50: 100%|██████████| 208/208 [1:16:04<00:00, 21.94s/it]


Epoch 2 - Loss: 674.0343, Train Acc: 41.67%
Test Acc: 41.15%


Epoch 3/50: 100%|██████████| 208/208 [1:16:01<00:00, 21.93s/it]


...

Epoch 48 - Loss: 538.7532, Train Acc: 56.41%
Test Acc: 52.15%


Epoch 49/50: 100%|██████████| 208/208 [1:18:32<00:00, 22.66s/it]


Epoch 49 - Loss: 503.5320, Train Acc: 58.81%
Test Acc: 56.46%


Epoch 50/50: 100%|██████████| 208/208 [1:18:33<00:00, 22.66s/it]


Epoch 50 - Loss: 500.8329, Train Acc: 59.29%
Test Acc: 57.89%
Modelo guardado como vgg16_3d_alzheimer3.pth


In [3]:
# Script para continuar entrenamiento VGG16 3D cargando pesos anteriores

import torch
import torch.nn as nn
import nibabel as nib
import pandas as pd
import numpy as np
import scipy.ndimage
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

CSV_PATH = r"D:/ADNI Data/BETandGroups.csv"
IMG_SIZE = (128, 128, 128)
BATCH_SIZE = 3
EPOCHS = 10  # ← entrenar 30 épocas más
LR = 1e-5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

# ========= DATASET =========
class Nifti3DDataset(Dataset):
    def __init__(self, df, label_encoder):
        self.df = df.reset_index(drop=True)
        self.label_encoder = label_encoder
        self.target_shape = IMG_SIZE

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.loc[idx, "Ruta"]
        label = self.df.loc[idx, "Grupo"]

        volume = nib.load(path).get_fdata().astype(np.float32)
        zoom_factors = [t / s for t, s in zip(self.target_shape, volume.shape)]
        volume = scipy.ndimage.zoom(volume, zoom=zoom_factors, order=3)
        volume = (volume - np.mean(volume)) / (np.std(volume) + 1e-8)
        volume = np.clip(volume, -3, 3)

        volume = torch.tensor(volume).unsqueeze(0)  # [1, D, H, W]
        label = torch.tensor(self.label_encoder.transform([label])[0])
        return volume, label

# ========= MODELO =========
class VGG3D(nn.Module):
    def __init__(self, num_classes=3):
        super(VGG3D, self).__init__()
        self.features = nn.Sequential(
            nn.Conv3d(1, 64, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(64, 64, 3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(2, 2),

            nn.Conv3d(64, 128, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(128, 128, 3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(2, 2),

            nn.Conv3d(128, 256, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(256, 256, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(256, 256, 3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(2, 2),

            nn.Conv3d(256, 512, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, 3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(2, 2),

            nn.Conv3d(512, 512, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, 3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(2, 2)
        )
        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# ========= DATOS =========
df = pd.read_csv(CSV_PATH)
le = LabelEncoder()
df['labels'] = le.fit_transform(df['Grupo'])

df_train, df_val = train_test_split(df, test_size=0.25, stratify=df['labels'], random_state=42)
train_dataset = Nifti3DDataset(df_train, le)
val_dataset = Nifti3DDataset(df_val, le)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# ========= CARGAR MODELO =========
model = VGG3D(num_classes=len(le.classes_)).to(DEVICE)
model.load_state_dict(torch.load("vgg16_3d_alzheimer3_cont.pth", map_location=DEVICE))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, min_lr=1e-8, verbose=True)

print("✅ Modelo cargado. Continuando entrenamiento...")

# ========= ENTRENAMIENTO =========
for epoch in range(EPOCHS):
    model.train()
    train_loss, train_correct = 0.0, 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()

    acc_train = 100 * train_correct / len(train_dataset)
    avg_train_loss = train_loss / len(train_dataset)
    print(f"✅ Epoch {epoch+1} - Train Loss: {avg_train_loss:.4f}, Train Acc: {acc_train:.2f}%")

    # Validación
    model.eval()
    val_correct, val_loss = 0, 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            val_correct += (outputs.argmax(1) == labels).sum().item()

    acc_val = 100 * val_correct / len(val_dataset)
    avg_val_loss = val_loss / len(val_dataset)
    print(f"🔍 Val Loss: {avg_val_loss:.4f}, Val Acc: {acc_val:.2f}%")

    scheduler.step(avg_val_loss)

# ========= GUARDAR =========
torch.save(model.state_dict(), "vgg16_3d_alzheimer4_cont.pth")
print("💾 Modelo actualizado guardado como vgg16_3d_alzheimer4_cont.pth")


d:\ADNI Data\venv\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


✅ Modelo cargado. Continuando entrenamiento...


Epoch 1/10: 100%|██████████| 208/208 [59:06<00:00, 17.05s/it]


✅ Epoch 1 - Train Loss: 0.6169, Train Acc: 74.84%
🔍 Val Loss: 0.7007, Val Acc: 66.51%


Epoch 2/10: 100%|██████████| 208/208 [1:02:09<00:00, 17.93s/it]


...

✅ Epoch 8 - Train Loss: 0.5575, Train Acc: 77.24%
🔍 Val Loss: 0.6512, Val Acc: 73.68%


Epoch 9/10: 100%|██████████| 208/208 [1:02:45<00:00, 18.10s/it]


✅ Epoch 9 - Train Loss: 0.4859, Train Acc: 79.33%
🔍 Val Loss: 0.5839, Val Acc: 74.16%


Epoch 10/10: 100%|██████████| 208/208 [1:03:32<00:00, 18.33s/it]


✅ Epoch 10 - Train Loss: 0.4382, Train Acc: 82.37%
🔍 Val Loss: 0.6163, Val Acc: 72.73%
💾 Modelo actualizado guardado como vgg16_3d_alzheimer4_cont.pth


In [4]:
# Script para continuar entrenamiento VGG16 3D cargando pesos anteriores

import torch
import torch.nn as nn
import nibabel as nib
import pandas as pd
import numpy as np
import scipy.ndimage
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

CSV_PATH = r"D:/ADNI Data/BETandGroups.csv"
IMG_SIZE = (128, 128, 128)
BATCH_SIZE = 3
EPOCHS = 10  # ← entrenar 30 épocas más
LR = 1e-5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

# ========= DATASET =========
class Nifti3DDataset(Dataset):
    def __init__(self, df, label_encoder):
        self.df = df.reset_index(drop=True)
        self.label_encoder = label_encoder
        self.target_shape = IMG_SIZE

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.loc[idx, "Ruta"]
        label = self.df.loc[idx, "Grupo"]

        volume = nib.load(path).get_fdata().astype(np.float32)
        zoom_factors = [t / s for t, s in zip(self.target_shape, volume.shape)]
        volume = scipy.ndimage.zoom(volume, zoom=zoom_factors, order=3)
        volume = (volume - np.mean(volume)) / (np.std(volume) + 1e-8)
        volume = np.clip(volume, -3, 3)

        volume = torch.tensor(volume).unsqueeze(0)  # [1, D, H, W]
        label = torch.tensor(self.label_encoder.transform([label])[0])
        return volume, label

# ========= MODELO =========
class VGG3D(nn.Module):
    def __init__(self, num_classes=3):
        super(VGG3D, self).__init__()
        self.features = nn.Sequential(
            nn.Conv3d(1, 64, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(64, 64, 3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(2, 2),

            nn.Conv3d(64, 128, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(128, 128, 3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(2, 2),

            nn.Conv3d(128, 256, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(256, 256, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(256, 256, 3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(2, 2),

            nn.Conv3d(256, 512, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, 3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(2, 2),

            nn.Conv3d(512, 512, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, 3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(2, 2)
        )
        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# ========= DATOS =========
df = pd.read_csv(CSV_PATH)
le = LabelEncoder()
df['labels'] = le.fit_transform(df['Grupo'])

df_train, df_val = train_test_split(df, test_size=0.25, stratify=df['labels'], random_state=42)
train_dataset = Nifti3DDataset(df_train, le)
val_dataset = Nifti3DDataset(df_val, le)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# ========= CARGAR MODELO =========
model = VGG3D(num_classes=len(le.classes_)).to(DEVICE)
model.load_state_dict(torch.load("vgg16_3d_alzheimer4_cont.pth", map_location=DEVICE))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, min_lr=1e-8, verbose=True)

print("✅ Modelo cargado. Continuando entrenamiento...")

# ========= ENTRENAMIENTO =========
for epoch in range(EPOCHS):
    model.train()
    train_loss, train_correct = 0.0, 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()

    acc_train = 100 * train_correct / len(train_dataset)
    avg_train_loss = train_loss / len(train_dataset)
    print(f"✅ Epoch {epoch+1} - Train Loss: {avg_train_loss:.4f}, Train Acc: {acc_train:.2f}%")

    # Validación
    model.eval()
    val_correct, val_loss = 0, 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            val_correct += (outputs.argmax(1) == labels).sum().item()

    acc_val = 100 * val_correct / len(val_dataset)
    avg_val_loss = val_loss / len(val_dataset)
    print(f"🔍 Val Loss: {avg_val_loss:.4f}, Val Acc: {acc_val:.2f}%")

    scheduler.step(avg_val_loss)

# ========= GUARDAR =========
torch.save(model.state_dict(), "vgg16_3d_alzheimer5_cont.pth")
print("💾 Modelo actualizado guardado como vgg16_3d_alzheimer4_cont.pth")


d:\ADNI Data\venv\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


✅ Modelo cargado. Continuando entrenamiento...


Epoch 1/10: 100%|██████████| 208/208 [1:03:10<00:00, 18.22s/it]


✅ Epoch 1 - Train Loss: 0.5002, Train Acc: 77.56%
🔍 Val Loss: 0.5524, Val Acc: 77.51%


Epoch 2/10: 100%|██████████| 208/208 [1:02:35<00:00, 18.05s/it]


...

✅ Epoch 8 - Train Loss: 0.3676, Train Acc: 84.46%
🔍 Val Loss: 0.5203, Val Acc: 82.78%


Epoch 9/10: 100%|██████████| 208/208 [1:02:57<00:00, 18.16s/it]


✅ Epoch 9 - Train Loss: 0.3634, Train Acc: 85.42%
🔍 Val Loss: 0.5044, Val Acc: 79.43%


Epoch 10/10: 100%|██████████| 208/208 [1:02:58<00:00, 18.17s/it]


✅ Epoch 10 - Train Loss: 0.3465, Train Acc: 86.70%
🔍 Val Loss: 0.4983, Val Acc: 83.25%
💾 Modelo actualizado guardado como vgg16_3d_alzheimer4_cont.pth


In [5]:
# Script para continuar entrenamiento VGG16 3D cargando pesos anteriores

import torch
import torch.nn as nn
import nibabel as nib
import pandas as pd
import numpy as np
import scipy.ndimage
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

CSV_PATH = r"D:/ADNI Data/BETandGroups.csv"
IMG_SIZE = (128, 128, 128)
BATCH_SIZE = 3
EPOCHS = 15  # ← entrenar 30 épocas más
LR = 1e-5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

# ========= DATASET =========
class Nifti3DDataset(Dataset):
    def __init__(self, df, label_encoder):
        self.df = df.reset_index(drop=True)
        self.label_encoder = label_encoder
        self.target_shape = IMG_SIZE

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.loc[idx, "Ruta"]
        label = self.df.loc[idx, "Grupo"]

        volume = nib.load(path).get_fdata().astype(np.float32)
        zoom_factors = [t / s for t, s in zip(self.target_shape, volume.shape)]
        volume = scipy.ndimage.zoom(volume, zoom=zoom_factors, order=3)
        volume = (volume - np.mean(volume)) / (np.std(volume) + 1e-8)
        volume = np.clip(volume, -3, 3)

        volume = torch.tensor(volume).unsqueeze(0)  # [1, D, H, W]
        label = torch.tensor(self.label_encoder.transform([label])[0])
        return volume, label

# ========= MODELO =========
class VGG3D(nn.Module):
    def __init__(self, num_classes=3):
        super(VGG3D, self).__init__()
        self.features = nn.Sequential(
            nn.Conv3d(1, 64, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(64, 64, 3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(2, 2),

            nn.Conv3d(64, 128, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(128, 128, 3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(2, 2),

            nn.Conv3d(128, 256, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(256, 256, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(256, 256, 3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(2, 2),

            nn.Conv3d(256, 512, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, 3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(2, 2),

            nn.Conv3d(512, 512, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, 3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, 3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(2, 2)
        )
        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# ========= DATOS =========
df = pd.read_csv(CSV_PATH)
le = LabelEncoder()
df['labels'] = le.fit_transform(df['Grupo'])

df_train, df_val = train_test_split(df, test_size=0.25, stratify=df['labels'], random_state=42)
train_dataset = Nifti3DDataset(df_train, le)
val_dataset = Nifti3DDataset(df_val, le)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# ========= CARGAR MODELO =========
model = VGG3D(num_classes=len(le.classes_)).to(DEVICE)
model.load_state_dict(torch.load("vgg16_3d_alzheimer5_cont.pth", map_location=DEVICE))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, min_lr=1e-8, verbose=True)

print("✅ Modelo cargado. Continuando entrenamiento...")

# ========= ENTRENAMIENTO =========
for epoch in range(EPOCHS):
    model.train()
    train_loss, train_correct = 0.0, 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()

    acc_train = 100 * train_correct / len(train_dataset)
    avg_train_loss = train_loss / len(train_dataset)
    print(f"✅ Epoch {epoch+1} - Train Loss: {avg_train_loss:.4f}, Train Acc: {acc_train:.2f}%")

    # Validación
    model.eval()
    val_correct, val_loss = 0, 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            val_correct += (outputs.argmax(1) == labels).sum().item()

    acc_val = 100 * val_correct / len(val_dataset)
    avg_val_loss = val_loss / len(val_dataset)
    print(f"🔍 Val Loss: {avg_val_loss:.4f}, Val Acc: {acc_val:.2f}%")

    scheduler.step(avg_val_loss)

# ========= GUARDAR =========
torch.save(model.state_dict(), "vgg16_3d_alzheimer6_cont.pth")
print("💾 Modelo actualizado guardado como vgg16_3d_alzheimer6_cont.pth")


d:\ADNI Data\venv\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


✅ Modelo cargado. Continuando entrenamiento...


Epoch 1/15: 100%|██████████| 208/208 [1:01:37<00:00, 17.78s/it]


✅ Epoch 1 - Train Loss: 0.3915, Train Acc: 83.97%
🔍 Val Loss: 0.5895, Val Acc: 76.56%


Epoch 2/15: 100%|██████████| 208/208 [1:01:53<00:00, 17.86s/it]


...

✅ Epoch 13 - Train Loss: 0.1673, Train Acc: 93.59%
🔍 Val Loss: 0.5115, Val Acc: 83.73%


Epoch 14/15: 100%|██████████| 208/208 [1:01:35<00:00, 17.77s/it]


✅ Epoch 14 - Train Loss: 0.1351, Train Acc: 95.03%
🔍 Val Loss: 0.5158, Val Acc: 84.21%


Epoch 15/15: 100%|██████████| 208/208 [1:01:37<00:00, 17.78s/it]


✅ Epoch 15 - Train Loss: 0.1227, Train Acc: 94.55%
🔍 Val Loss: 0.5536, Val Acc: 83.25%
💾 Modelo actualizado guardado como vgg16_3d_alzheimer6_cont.pth
